In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps
from osgeo import gdal
import geopandas as gpd
import json

In [2]:
m = leafmap.Map(draw_control=True, layers_control=True, measure_control=True, fullscreen_control=False, attribution_control=True)
m.add_basemap("SATELLITE")
# Load the geojson file
file_path = './combined_coordinates.geojson'
gdf = gpd.read_file(file_path)

# Extract coordinates
coordinates = [(point.y, point.x) for point in gdf.geometry]

# Calculate the center of all the points
center_lat = sum(point[0] for point in coordinates) / len(coordinates)
center_lon = sum(point[1] for point in coordinates) / len(coordinates)
center = (center_lat, center_lon)

# Create a map and add points as markers
m = leafmap.Map(center=center, zoom=10)
for lat, lon in coordinates:
    m.add_marker(location=(lat, lon))

m

Map(center=[41.215885328629255, 116.5437474543746], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [21]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# 读取 GeoJSON 文件
file_path = './combined_coordinates.geojson'
gdf = gpd.read_file(file_path)
gdf.to_crs(epsg=4548, inplace=True)

# 计算每个点的最近距离
distances = []

for i, point in enumerate(gdf.geometry):
    min_distance = float('inf')
    for j, other_point in enumerate(gdf.geometry):
        if i != j:
            distance = point.distance(other_point)
            if distance < min_distance:
                min_distance = distance
    distances.append(min_distance)

# 将最近距离添加到 GeoDataFrame 中
gdf['nearest_distance'] = distances

# 显示结果
gdf[['geometry', 'nearest_distance']]


,geometry,nearest_distance
0,POINT (436629.231 4490536.802),2498.663837
1,POINT (434734.441 4565175.559),4598.506549
2,POINT (430697.928 4544509.755),20986.442645
3,POINT (433213.239 4495997.227),1964.095961
4,POINT (434290.814 4494355.122),939.444555
...,...,...
259,POINT (540026.024 4633548.940),4715.277215
260,POINT (578523.887 4654319.910),7525.886236
261,POINT (571558.390 4659429.038),1481.270391
262,POINT (571961.145 4658003.573),1481.270391


In [22]:
gdf

,地点,经度,纬度,Table Name,geometry,nearest_distance
0,后店村,116.251900,40.547000,丰宁县-01,POINT (436629.231 4490536.802),2498.663837
1,富贵山村,116.221700,41.218900,丰宁县-02,POINT (434734.441 4565175.559),4598.506549
2,九宫号村,116.175900,41.032500,丰宁县-03,POINT (430697.928 4544509.755),20986.442645
3,高栅子村,116.211000,40.595900,丰宁县-04,POINT (433213.239 4495997.227),1964.095961
4,杨木栅子村,116.223900,40.581200,丰宁县-05,POINT (434290.814 4494355.122),939.444555
...,...,...,...,...,...,...
259,木兰林场北沟分场黄家东沟,117.481875,41.836140,木兰围场林场-41,POINT (540026.024 4633548.940),4715.277215
260,木兰林场八英庄分场光顶山林路旁,117.948071,42.020240,木兰围场林场-42,POINT (578523.887 4654319.910),7525.886236
261,木兰林场八英庄分场贾家梁,117.864605,42.066896,木兰围场林场-43,POINT (571558.390 4659429.038),1481.270391
262,木兰林场八英庄分场下窝铺,117.869296,42.054027,木兰围场林场-44,POINT (571961.145 4658003.573),1481.270391


In [34]:
gdf['nearest_distance'].max()

inf